In [3]:
import pandas as pd
import numpy as np
import os

np.random.seed(42)

NUM_ORDERS = 200_000

orders = pd.DataFrame({
    "order_id": range(1, NUM_ORDERS + 1),
    "customer_id": np.random.randint(1, 50001, NUM_ORDERS),
    "product_id": np.random.randint(1, 5001, NUM_ORDERS),
    "order_date": pd.to_datetime(
        np.random.choice(
            pd.date_range("2022-01-01", "2023-12-31"),
            NUM_ORDERS
        )
    ),
    "quantity": np.random.randint(1, 5, NUM_ORDERS),
    "discount": np.round(np.random.uniform(0, 0.5, NUM_ORDERS), 2),
    "order_status": np.random.choice(
        ["Delivered", "Cancelled", "Returned"],
        size=NUM_ORDERS,
        p=[0.85, 0.1, 0.05]
    )
})

# 🔴 THIS IS THE LINE THAT CREATES THE FILE
os.makedirs("data/raw", exist_ok=True)
orders.to_csv("data/raw/orders.csv", index=False)

print("orders.csv CREATED successfully")
orders.head()

orders.csv CREATED successfully


,order_id,customer_id,product_id,order_date,quantity,discount,order_status
0,1,15796,2346,2022-06-09,3,0.40,Delivered
1,2,861,989,2022-05-15,4,0.42,Delivered
2,3,38159,422,2023-10-26,3,0.02,Delivered
3,4,44733,1945,2022-02-11,1,0.24,Delivered
4,5,11285,878,2023-01-18,3,0.24,Delivered


In [4]:
import pandas as pd
import os

# Step 1: Load raw orders file (CONFIRMED PATH)
orders = pd.read_csv("data/raw/orders.csv")

# Step 2: Basic cleaning
orders["order_date"] = pd.to_datetime(orders["order_date"])
orders = orders.drop_duplicates()

orders["quantity"] = orders["quantity"].astype(int)
orders["discount"] = orders["discount"].astype(float)

orders.head()


,order_id,customer_id,product_id,order_date,quantity,discount,order_status
0,1,15796,2346,2022-06-09,3,0.40,Delivered
1,2,861,989,2022-05-15,4,0.42,Delivered
2,3,38159,422,2023-10-26,3,0.02,Delivered
3,4,44733,1945,2022-02-11,1,0.24,Delivered
4,5,11285,878,2023-01-18,3,0.24,Delivered


In [5]:
# Step 3: Create processed folder (core-level)
os.makedirs("../data/processed", exist_ok=True)

# Step 4: Save cleaned data
orders.to_csv(
    "../data/processed/orders_cleaned.csv",
    index=False
)

print("✅ orders_cleaned.csv created successfully")


✅ orders_cleaned.csv created successfully


In [6]:
pd.read_csv("../data/processed/orders_cleaned.csv").head()

,order_id,customer_id,product_id,order_date,quantity,discount,order_status
0,1,15796,2346,2022-06-09,3,0.40,Delivered
1,2,861,989,2022-05-15,4,0.42,Delivered
2,3,38159,422,2023-10-26,3,0.02,Delivered
3,4,44733,1945,2022-02-11,1,0.24,Delivered
4,5,11285,878,2023-01-18,3,0.24,Delivered


In [7]:
import pandas as pd

orders = pd.read_csv("../data/processed/orders_cleaned.csv")

orders["order_date"] = pd.to_datetime(orders["order_date"])
orders.head()

,order_id,customer_id,product_id,order_date,quantity,discount,order_status
0,1,15796,2346,2022-06-09,3,0.40,Delivered
1,2,861,989,2022-05-15,4,0.42,Delivered
2,3,38159,422,2023-10-26,3,0.02,Delivered
3,4,44733,1945,2022-02-11,1,0.24,Delivered
4,5,11285,878,2023-01-18,3,0.24,Delivered


In [8]:
orders["order_month"] = orders["order_date"].dt.to_period("M").astype(str)

orders[["customer_id", "order_month"]].head()

,customer_id,order_month
0,15796,2022-06
1,861,2022-05
2,38159,2023-10
3,44733,2022-02
4,11285,2023-01


In [9]:
monthly_customer = (
    orders
    .groupby(["customer_id", "order_month"])
    .agg(
        monthly_orders=("order_id", "count"),
        monthly_quantity=("quantity", "sum"),
        avg_discount=("discount", "mean")
    )
    .reset_index()
)

monthly_customer.head()

,customer_id,order_month,monthly_orders,monthly_quantity,avg_discount
0,1,2022-02,1,4,0.08
1,1,2023-12,1,1,0.08
2,2,2022-02,1,4,0.12
3,2,2022-10,1,1,0.49
4,2,2022-12,1,1,0.39


In [11]:
monthly_customer["monthly_value"] = (
    monthly_customer["monthly_quantity"] *
    (1 - monthly_customer["avg_discount"])
)

monthly_customer.head()

,customer_id,order_month,monthly_orders,monthly_quantity,avg_discount,monthly_value
0,1,2022-02,1,4,0.08,3.68
1,1,2023-12,1,1,0.08,0.92
2,2,2022-02,1,4,0.12,3.52
3,2,2022-10,1,1,0.49,0.51
4,2,2022-12,1,1,0.39,0.61


In [12]:
monthly_customer.to_csv(
    "../data/processed/customer_monthly_value.csv",
    index=False
)

print("✅ customer_monthly_value.csv created successfully")

✅ customer_monthly_value.csv created successfully


In [13]:
import pandas as pd
import numpy as np

monthly = pd.read_csv("../data/processed/customer_monthly_value.csv")

monthly["order_month"] = pd.to_datetime(monthly["order_month"])
monthly.head()

C:\Users\yashm\AppData\Local\Temp\ipykernel_18308\1286511385.py:6: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  monthly["order_month"] = pd.to_datetime(monthly["order_month"])


,customer_id,order_month,monthly_orders,monthly_quantity,avg_discount,monthly_value
0,1,2022-02-01,1,4,0.08,3.68
1,1,2023-12-01,1,1,0.08,0.92
2,2,2022-02-01,1,4,0.12,3.52
3,2,2022-10-01,1,1,0.49,0.51
4,2,2022-12-01,1,1,0.39,0.61


In [14]:
from sklearn.linear_model import LinearRegression

In [15]:
def compute_trend(values):
    if len(values) < 2:
        return 0.0
    X = np.arange(len(values)).reshape(-1, 1)
    y = values.reshape(-1, 1)
    model = LinearRegression().fit(X, y)
    return model.coef_[0][0]


In [16]:
# Reference date = last month in data
ref_date = monthly["order_month"].max()

customer_clv_features = (
    monthly
    .sort_values(["customer_id", "order_month"])
    .groupby("customer_id")
    .agg(
        total_value=("monthly_value", "sum"),
        avg_monthly_value=("monthly_value", "mean"),
        value_std=("monthly_value", "std"),
        active_months=("order_month", "nunique"),
        last_active=("order_month", "max"),
        trend=("monthly_value", lambda x: compute_trend(x.values))
    )
    .reset_index()
)

customer_clv_features["recency_months"] = (
    (ref_date - customer_clv_features["last_active"]).dt.days // 30
)

customer_clv_features.drop(columns=["last_active"], inplace=True)

customer_clv_features.head()


,customer_id,total_value,avg_monthly_value,value_std,active_months,trend,recency_months
0,1,4.60,2.300000,1.951615,2,-2.760000,0
1,2,13.86,1.980000,1.246916,7,-0.048571,0
2,3,14.84,2.473333,1.005777,6,0.289143,0
3,4,9.14,1.523333,1.184883,6,0.192000,1
4,5,3.70,1.850000,0.438406,2,0.620000,17


In [17]:
customer_clv_features["value_std"] = customer_clv_features["value_std"].fillna(0.0)
customer_clv_features["trend"] = customer_clv_features["trend"].fillna(0.0)

customer_clv_features.describe()

,customer_id,total_value,avg_monthly_value,value_std,active_months,trend,recency_months
count,49120.000000,49120.000000,49120.000000,49120.000000,49120.000000,49120.000000,49120.000000
mean,25001.133286,7.639353,2.036466,0.900367,3.751059,0.000220,5.054479
std,14432.017974,4.093398,0.665516,0.568809,1.699425,0.793906,4.988295
min,1.000000,0.500000,0.500000,0.000000,1.000000,-6.556667,0.000000
25%,12510.750000,4.578750,1.601667,0.539290,2.000000,-0.297214,1.000000
50%,24999.500000,7.140000,1.990000,0.877154,4.000000,0.000000,4.000000
75%,37510.250000,10.120000,2.408000,1.198133,5.000000,0.295000,7.000000
max,50000.000000,31.585000,9.020000,5.744064,12.000000,8.123333,23.000000


In [18]:
customer_clv_features.to_csv(
    "../data/processed/customer_clv_features.csv",
    index=False
)

print("✅ customer_clv_features.csv created successfully")


✅ customer_clv_features.csv created successfully


In [19]:
import pandas as pd

monthly = pd.read_csv("../data/processed/customer_monthly_value.csv")
clv_features = pd.read_csv("../data/processed/customer_clv_features.csv")
dashboard = pd.read_csv("../data/processed/dashboard_data.csv")

monthly["order_month"] = pd.to_datetime(monthly["order_month"])

monthly.head(), clv_features.head(), dashboard.head()

C:\Users\yashm\AppData\Local\Temp\ipykernel_18308\1870784294.py:7: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  monthly["order_month"] = pd.to_datetime(monthly["order_month"])


(   customer_id order_month  monthly_orders  monthly_quantity  avg_discount  \
 0            1  2022-02-01               1                 4          0.08   
 1            1  2023-12-01               1                 1          0.08   
 2            2  2022-02-01               1                 4          0.12   
 3            2  2022-10-01               1                 1          0.49   
 4            2  2022-12-01               1                 1          0.39   
 
    monthly_value  
 0           3.68  
 1           0.92  
 2           3.52  
 3           0.51  
 4           0.61  ,
    customer_id  total_value  avg_monthly_value  value_std  active_months  \
 0            1         4.60           2.300000   1.951615              2   
 1            2        13.86           1.980000   1.246916              7   
 2            3        14.84           2.473333   1.005777              6   
 3            4         9.14           1.523333   1.184883              6   
 4            5   

In [20]:
customer_segments = dashboard[["customer_id", "segment"]].drop_duplicates()

monthly = monthly.merge(
    customer_segments,
    on="customer_id",
    how="left"
)

monthly.head()

,customer_id,order_month,monthly_orders,monthly_quantity,avg_discount,monthly_value,segment
0,1,2022-02-01,1,4,0.08,3.68,NaN
1,1,2023-12-01,1,1,0.08,0.92,NaN
2,2,2022-02-01,1,4,0.12,3.52,NaN
3,2,2022-10-01,1,1,0.49,0.51,NaN
4,2,2022-12-01,1,1,0.39,0.61,NaN


In [21]:
segment_clv_dynamics = (
    monthly
    .groupby(["segment", "order_month"])
    .agg(
        segment_value=("monthly_value", "sum"),
        active_customers=("customer_id", "nunique"),
        avg_value_per_customer=("monthly_value", "mean")
    )
    .reset_index()
)

segment_clv_dynamics.head()

,segment,order_month,segment_value,active_customers,avg_value_per_customer
0,0.0,2022-01-01,432.970,222,1.950315
1,0.0,2022-02-01,356.090,192,1.854635
2,0.0,2022-03-01,360.050,199,1.809296
3,0.0,2022-04-01,299.700,163,1.838650
4,0.0,2022-05-01,266.565,155,1.719774


In [22]:
segment_clv_dynamics.isnull().sum()

segment                   0
order_month               0
segment_value             0
active_customers          0
avg_value_per_customer    0
dtype: int64

In [23]:
segment_clv_dynamics.describe()

,segment,order_month,segment_value,active_customers,avg_value_per_customer
count,96.000000,96,96.000000,96.000000,96.000000
mean,1.500000,2022-12-16 00:00:00,773.589722,381.166667,1.951830
min,0.000000,2022-01-01 00:00:00,4.060000,6.000000,0.676667
25%,0.750000,2022-06-23 12:00:00,288.993750,161.000000,1.852530
50%,1.500000,2022-12-16 12:00:00,794.340000,399.500000,1.937249
75%,2.250000,2023-06-08 12:00:00,1263.059583,591.000000,2.070088
max,3.000000,2023-12-01 00:00:00,1599.960000,825.000000,2.303438
std,1.123903,NaN,517.367158,246.477357,0.216474


In [24]:
segment_clv_dynamics.to_csv(
    "../data/processed/segment_clv_dynamics.csv",
    index=False
)

print("✅ segment_clv_dynamics.csv created successfully")

✅ segment_clv_dynamics.csv created successfully
